# Simple NN Regression Model
**Goal**: Use metrological data for the year to predict the crop yield for that year.

**Run instructions**:
- in collab, change ``GOOGLE_DRIVE_PATH`` to appropriate directory structure (marked with ``# TODO``)
- select crop to predict yield for by changing crop variable in data download section (marked with ``# TODO``)
- *optional*: update hyperparameters in section ``# Set hyper-parameters``
- then just select 'Run all', results + plots will print at bottom

**Possible architecture improvments**:
- Add more layers to simple NN
- Try transformer architecture

**Future code improvments**:
- Add Torchvision experiment tracking, etc.
- Model weight visualization- heat map to see which meterological factors in which month are most important to each crop.
- Refactor code: functionalize hyper-parameter tuning, save best performing-model, etc.

**Improvements attempted**
1. Combine all crops into 1 data set with dummy variables for crop type => intial results worse (lower RMSE, ~26 without tuning)
2. Add ``asd_desc`` which describes the type of land in each county (make dummy variables, etc.) => helps very slightly (e.g. for Corn, RMSE goes down  1, to ~15 from ~16)
3. fine-tune hyperparameters (learning rate, hidden dimensions, batch size, weight decay, etc.) => random search better + faster than grid but neither much better than intuition (grid search actually worse when only given 10 epochs instead of 20)
  - to run grid search, set ```do_grid_search=true```
  - to run random search, set ```NUM_SEARCH``` to postive integer (e.g. 10)

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from google.colab import drive
import os
drive.mount('/content/drive')
GOOGLE_DRIVE_PATH_POST_MYDRIVE = 'DL/MMST-ViT-main'  # TODO change this
GOOGLE_DRIVE_PATH = os.path.join('/content', 'drive', 'MyDrive', GOOGLE_DRIVE_PATH_POST_MYDRIVE)
print(os.listdir(GOOGLE_DRIVE_PATH))

In [ ]:
import sys
sys.path.append(GOOGLE_DRIVE_PATH)

if 'google.colab' in sys.modules:
  print(f'Running in google colab. Our path is `{GOOGLE_DRIVE_PATH}`')
else:
  GOOGLE_DRIVE_PATH = '.'
  print('Running locally.')

In [ ]:
import random
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import ParameterGrid

# Pytorch package
import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torchvision import transforms
from typing import Tuple, Dict, List

# Tqdm progress bar
from tqdm import tqdm_notebook, tqdm

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("You are using device: %s" % device)

In [ ]:
seed = 123
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
np.random.seed(seed)

In [ ]:
# load data - 1 crop
crop = 'Cotton' # TODO set as appropriate. Options= Corn, Cotton, Soybeans, WinterWheat
# 'region' will be 1 crop but includes include dummy columns for region,
# 'all_crops' has all 4 crops (identified by dummy cols) in 1 dataset
# default is 1 crop without dummy columns for region
dataset = True # TODO set as desired. Options = 'region', 'all_crops', 'default'

if dataset == 'region':
    df = pd.read_csv(f'{GOOGLE_DRIVE_PATH}/simple_model/monthly_{crop}_with_region.csv')
elif dataset == 'all_crops':
    df = pd.read_csv(f'{GOOGLE_DRIVE_PATH}/simple_model/all_crops.csv')
else:
  df = pd.read_csv(f'{GOOGLE_DRIVE_PATH}/simple_model/monthly_{crop}_model_data.csv')

# # to load 1 dataset with all 4 crops, identified by dummy columns
# df = pd.read_csv(f'{GOOGLE_DRIVE_PATH}/simple_model/all_crops.csv')

df.head()
X = df.drop(columns='yield').to_numpy()
y = df['yield'].to_numpy()

num_data_features = X.shape[1]
X.shape, y.shape, num_data_features

In [ ]:
# train-test split of the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, shuffle=True)

# Standardizing data
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)


X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32).reshape(-1, 1)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32).reshape(-1, 1)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)
print(f'{X_train[0].shape=}')
print(f'{y_train[0]=}')

In [ ]:
# turn into correct type format to be put into data loader
# based on pytorch docs
class CustomDataset(Dataset):
    def __init__(self, X_p, y_p, transform=None, target_transform=None):
        self.X = X_p
        self.y = y_p
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, idx):
        item = self.X[idx]
        label = self.y[idx]
        if self.transform:
            item = self.transform(item)
        if self.target_transform:
            label = self.target_transform(label)
        return item, label

train_data = CustomDataset(X_train, y_train)
test_data = CustomDataset(X_test, y_test)

In [ ]:
from torch.utils.data import DataLoader
# Setup batch size and number of workers
BATCH_SIZE = 4
NUM_WORKERS = os.cpu_count()

train_dataloader = DataLoader(dataset=train_data, # use custom created train Dataset
                                     batch_size=BATCH_SIZE, # how many samples per batch?
                                     num_workers=NUM_WORKERS, # how many subprocesses to use for data loading? (higher = more)
                                     shuffle=True) # shuffle the data?

test_dataloader = DataLoader(dataset=test_data, # use custom created test Dataset
                                    batch_size=BATCH_SIZE,
                                    num_workers=NUM_WORKERS,
                                    shuffle=False) # don't usually need to shuffle testing data

train_dataloader, test_dataloader
next(iter(train_dataloader))
# # check ok
item, label = next(iter(train_dataloader))
print(f"Data shape: {item.shape} -> [batch_size, # features]")
print(f"Label shape: {label.shape}")

In [ ]:
class MyModel(nn.Module):
    """
    Model architecture is just basic to test code
    """
    def __init__(self, input_features: int, hidden_units: int, act_func=nn.ReLU()) -> None:
        super().__init__()
        self.linear_layer_stack = nn.Sequential(
            nn.Linear(in_features=input_features, out_features=hidden_units),
            act_func,
            # nn.Linear(in_features=hidden_units, out_features=hidden_units),
            # act_func,
            nn.Linear(in_features=hidden_units, out_features=1)
        )

    def forward(self, x):
        return self.linear_layer_stack(x)

In [ ]:
# Training code adapted from: https://www.learnpytorch.io/

def train_step(model: torch.nn.Module,
               dataloader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer):
    # Put model in train mode
    model.train()

    # Setup train loss and train accuracy values
    train_loss, train_acc = 0, 0

    # Loop through data loader data batches
    for batch, (X, y) in enumerate(dataloader):
        # Send data to target device
        X, y = X.to(device), y.to(device)

        # 1. Forward pass
        y_pred = model(X)

        # 2. Calculate  and accumulate loss
        loss = loss_fn(y_pred, y)
        train_loss += loss.item()

        # 3. Optimizer zero grad
        optimizer.zero_grad()

        # 4. Loss backward
        loss.backward()

        # 5. Optimizer step
        optimizer.step()


    # Adjust metrics to get average loss and accuracy per batch
    train_loss = train_loss / len(dataloader)

    return train_loss

In [ ]:
# Training code adapted from: https://www.learnpytorch.io/
def test_step(model: torch.nn.Module,
              dataloader: torch.utils.data.DataLoader,
              loss_fn: torch.nn.Module):
    # Put model in eval mode
    model.eval()

    # Setup test loss and test accuracy values
    test_loss, test_acc = 0, 0

    # Turn on inference context manager
    with torch.inference_mode():
        # Loop through DataLoader batches
        for batch, (X, y) in enumerate(dataloader):
            # Send data to target device
            X, y = X.to(device), y.to(device)

            # 1. Forward pass
            test_pred_logits = model(X)

            # 2. Calculate and accumulate loss
            loss = loss_fn(test_pred_logits, y)
            test_loss += loss.item()

    # Adjust metrics to get average loss and accuracy per batch
    test_loss = test_loss / len(dataloader)
    return test_loss

In [ ]:
# Training code adapted from: https://www.learnpytorch.io/
from tqdm.auto import tqdm

# 1. Take in various parameters required for training and test steps
def train(model: torch.nn.Module,
          train_dataloader: torch.utils.data.DataLoader,
          test_dataloader: torch.utils.data.DataLoader,
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module = nn.CrossEntropyLoss(),
          epochs: int = 5):

    # 2. Create empty results dictionary
    results = {"train_loss": [],
        "test_loss": [],
    }

    # 3. Loop through training and testing steps for a number of epochs
    for epoch in tqdm(range(epochs)):
        train_loss = train_step(model=model,
                                           dataloader=train_dataloader,
                                           loss_fn=loss_fn,
                                           optimizer=optimizer)
        test_loss = test_step(model=model, dataloader=test_dataloader, loss_fn=loss_fn)

        # # 4. Print out what's happening
        # if epoch % 5 == 0 or epoch == epochs - 1:
        print(
            f"Epoch: {epoch} | "
            f"train_loss: {train_loss:.4f} | "
            f"test_loss: {test_loss:.4f}"
        )

        # 5. Update results dictionary
        results["train_loss"].append(train_loss)
        results["test_loss"].append(test_loss)

    # 6. Return the filled results at the end of the epochs
    return results

In [ ]:
# based on: https://gist.github.com/jamesr2323/33c67ba5ac29880171b63d2c7f1acdc5
class RMSELoss(torch.nn.Module):
    def __init__(self):
        super(RMSELoss,self).__init__()

    def forward(self,x,y):
        criterion = nn.MSELoss()
        eps = 1e-6
        loss = torch.sqrt(criterion(x, y) + eps)
        return loss

In [ ]:
# Plotting code adapted from: https://www.learnpytorch.io/
import matplotlib.pyplot as plt

def plot_loss_curves(results: Dict[str, List[float]]):
    """Plots training curves of a results dictionary.

    Args:
        results (dict): dictionary containing list of values, e.g.
            {"train_loss": [...],
             "test_loss": [...],
    """

    # Get the loss values of the results dictionary (training and test)
    loss = results['train_loss']
    test_loss = results['test_loss']

    # Figure out how many epochs there were
    epochs = range(len(results['train_loss']))

    # Setup a plot
    plt.figure(figsize=(15, 7))

    # Plot loss
    plt.subplot(1, 2, 1)
    plt.plot(epochs, loss, label='train_loss')
    plt.plot(epochs, test_loss, label='test_loss')
    plt.title(f'Loss: {crop}')
    plt.xlabel('Epochs')
    plt.legend()

In [ ]:
# Set random seeds
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

# Set hyper-parameters
NUM_EPOCHS = 20
HIDDEN_DIM = 45 # corn: 80, cotton: 64, soybeans:80/55 winterwheat 80/65
learning_rate = 0.01 # corn: 0.001, cotton: 0.01 , soybeans 0.005, winterwheat  0.05
weight_decay= 0.05 # corn: 0.01, cotton: 0.1, spybeans: 0.01, winterwheat 0.05
activation_func = nn.LeakyReLU() #corn: nn.LeakyReLU(), cotton: nn.SELU(), soybeans: nn.ReLU()
# Setup model
model_0 = MyModel(num_data_features, HIDDEN_DIM, activation_func).to(device)

# Setup loss function and optimizer
loss_fn = RMSELoss() # nn.MSELoss()
optimizer = torch.optim.Adam(params=model_0.parameters(), lr=learning_rate, weight_decay=weight_decay)

# Start the timer
from timeit import default_timer as timer
start_time = timer()

# Train model_0
model_0_results = train(model=model_0,
                        train_dataloader=train_dataloader,
                        test_dataloader=test_dataloader,
                        optimizer=optimizer,
                        loss_fn=loss_fn,
                        epochs=NUM_EPOCHS)

# End the timer and print out how long it took
end_time = timer()
print(f"Total training time: {end_time-start_time:.3f} seconds")
plot_loss_curves(model_0_results)

In [ ]:
# GRID SEARCH
do_grid_search = False
NUM_EPOCHS = 10
activation_func = nn.LeakyReLU()

hyperparam_grid = {
    'HIDDEN_DIM': [20, 64, 100],
    'learning_rate': [1e-2, 1e-3, 5e-3, 1e-4],
    'weight_decay': [0.1, 0.01, 0.05, 0.001],
    # 'activation_func': [nn.LeakyReLU(), nn.ReLU(), nn.SELU(), nn.GELU()]
}

if do_grid_search:
  best_rmse = float('inf')
  best_config = None
  best_rmse_last_epoch = float('inf')
  best_config_last_epoch = None
  for config in ParameterGrid(hyperparam_grid):
    print(f'Training with config: {config}')
    model_0 = MyModel(num_data_features, config['HIDDEN_DIM'], activation_func).to(device)

    # Setup loss function and optimizer
    loss_fn = RMSELoss() # nn.MSELoss()
    optimizer = torch.optim.Adam(params=model_0.parameters(), lr=config['learning_rate'], weight_decay=config['weight_decay'])

    # Start the timer
    from timeit import default_timer as timer
    start_time = timer()

    # Train model_0
    model_0_results = train(model=model_0,
                            train_dataloader=train_dataloader,
                            test_dataloader=test_dataloader,
                            optimizer=optimizer,
                            loss_fn=loss_fn,
                            epochs=NUM_EPOCHS)
    final_rmse = model_0_results['test_loss'][-1]
    min_rmse = min(model_0_results['test_loss'])
    if min_rmse < best_rmse:
        best_rmse = min_rmse
        best_config = config
    if final_rmse < best_rmse_last_epoch:
        best_rmse_last_epoch = final_rmse
        best_config_last_epoch = config

  print(f'Best RMSE: {best_rmse}, Best Config: {best_config}')
  print(f'Best RMSE at last epoch: {best_rmse_last_epoch}, Best Config at last epoch: {best_config_last_epoch}')
else:
  print('Set train_grid=True if you want to run the grid search. See cell above for results of 1 model.')


In [ ]:
# RANDOM SEARCH
# If set NUM_SEARCH to 0, won't run
NUM_SEARCH = 30

NUM_EPOCHS = 20
learning_rate_range = result = [x / 10000 for x in range(1, 1000, 10)]
weight_decay_rate_range = result = [x / 100000 for x in range(1, 10000, 10)]
hidden_dim_range = result = range(20, 100, 5)
activation_functions = [nn.LeakyReLU(), nn.ReLU(), nn.SELU(), nn.GELU()]

best_rmse = float('inf')
best_config = None
best_rmse_last_epoch = float('inf')
best_config_last_epoch = None

for i in range(NUM_SEARCH):
  learning_rate = random.choice(learning_rate_range)
  weight_decay = random.choice(weight_decay_rate_range)
  hidden_dim = random.choice(hidden_dim_range)
  activation_func = random.choice(activation_functions)
  config = {"learning_rate": learning_rate, 'weight_decay': weight_decay,
            'hidden_dim': hidden_dim, 'activation_func': activation_func }
  print(f"Run {i}: config is {config}")

  # Setup model
  model_0 = MyModel(num_data_features, hidden_dim, activation_func).to(device)

  # Setup loss function and optimizer
  loss_fn = RMSELoss() # nn.MSELoss()
  optimizer = torch.optim.Adam(params=model_0.parameters(), lr=learning_rate, weight_decay=weight_decay)

  # Train model_0
  model_0_results = train(model=model_0,
                          train_dataloader=train_dataloader,
                          test_dataloader=test_dataloader,
                          optimizer=optimizer,
                          loss_fn=loss_fn,
                          epochs=NUM_EPOCHS)

  print(model_0_results)

  final_rmse = model_0_results['test_loss'][-1]
  min_rmse = min(model_0_results['test_loss'])
  if min_rmse < best_rmse:
      best_rmse = min_rmse
      best_config = config
  if final_rmse < best_rmse_last_epoch:
      best_rmse_last_epoch = final_rmse
      best_config_last_epoch = config

print(f'Best RMSE: {best_rmse}, Best Config: {best_config}')
print(f'Best RMSE at last epoch: {best_rmse_last_epoch}, Best Config at last epoch: {best_config_last_epoch}')





In [ ]:
print('Summary stats for paper')
print(f'{crop=}, {min(y)=}, {max(y)=}, {np.mean(y)=}, {np.std(y)=}')
corn = pd.read_csv(f'{GOOGLE_DRIVE_PATH}/simple_model/monthly_Corn_with_region.csv')
print(f'{corn.shape=}')
corn = pd.read_csv(f'{GOOGLE_DRIVE_PATH}/simple_model/monthly_Corn_model_data.csv')
print(f'{corn.shape=}')
print(f"Number region cols: {len([col for col in corn.columns if 'asd_desc' in col])}")